---
latex-output-dir: output
execute:
  echo: false
---
















# Journals {#appendix-h}

\fontsize{9}{10}\selectfont


In [ ]:
# 
import bibtexparser
import requests
from IPython.display import Markdown
from tabulate import tabulate

file_path = '/Users/admin/Projects/literature/phd_kul/bib/bibliography_citedrive.bib'
#file_path = '/Users/admin/Projects/literature/phd_kul/bib/MySelection.bib'

<!-- add journal impact factor (h-score) google scholar scraping? 
merk op Software uitgever is MDPI niet Elsevier, beter zou zijn publisher ophalen
aan de hand van DOI -->


In [ ]:
#| label: tbl-journals
#| tbl-cap: Journals

def extract_journals_from_bibtex(file_path):
    with open(file_path) as bibtex_file:
        bibtex_str = bibtex_file.read()

    bib_database = bibtexparser.loads(bibtex_str)
    journals = set()

    for entry in bib_database.entries:
        if entry.get('ENTRYTYPE') == 'article' and 'journal' in entry:
            journals.add(entry['journal'])

    sorted_journals = sorted(journals)
    return sorted_journals


def find_journal_info(journal_title):
    # Initialize an empty list to hold the result
    result = []

    # Encode the journal title for use in a URL
    encoded_title = requests.utils.quote(journal_title)

    # Construct the CrossRef API request URL
    api_url = f"https://api.crossref.org/journals?query={encoded_title}"

    # Make the GET request
    response = requests.get(api_url)

    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Check if there are any results
        if data['message']['total-results'] > 0:
            # Assuming the first result is the most relevant one
            first_journal = data['message']['items'][0]
            publisher = first_journal.get('publisher', 'Publisher not found')
            # Journals can have multiple ISSNs; we'll collect them all
            issn = first_journal.get('ISSN', ['ISSN not found'])[0]
        
            # Append the journal information to the result list
            result = [journal_title, publisher, issn]

        # else:
        #    return []
    else:
        print("Failed to retrieve data from CrossRef API.")
    
    return result

header = ["#", "Journal", "Publisher", "ISSN"]
table = []
journals = extract_journals_from_bibtex(file_path)
for idx, journal in enumerate(journals, start=1):
    journal_info = find_journal_info(journal)
    if journal_info != []:
        row = [idx] + journal_info
        table.append(row)
Markdown(tabulate(table, header, maxcolwidths=[2, 40, 35, 10]))

The publisher and the ISSN number have been looked up at [crossref](https://www.crossref.org) using the journal name in the bibtex file.


In [ ]:
#| label: tbl-authors
#| tbl-cap: Authors

def extract_authors_from_bibtex(file_path):
    with open(file_path) as bibtex_file:
        bibtex_str = bibtex_file.read()

    bib_database = bibtexparser.loads(bibtex_str)
    authors_list = []

    for entry in bib_database.entries:
        if 'author' in entry:
            entry_authors = [author.strip() for author in entry['author'].split(' and ')]
            first_author = entry_authors[0]  # Extract the first author
            other_authors = ', '.join(entry_authors[1:])  # Combine the remaining authors
            authors_list.append((first_author, other_authors))  # Append as a tuple

    # No need to sort here as tabulate will handle the display
    return authors_list

authors = extract_authors_from_bibtex(file_path)

# Format and print the table in Markdown format
Markdown(tabulate(authors, headers=["First Author", "Other Authors"], maxcolwidths=[40, 50]))

\newpage


In [ ]:
#| label: tbl-keywords
#| tbl-cap: Keywords

def extract_keys_and_keywords(file_path):
    with open(file_path) as bibtex_file:
        bibtex_str = bibtex_file.read()

    bib_database = bibtexparser.loads(bibtex_str)
    data = []

    for entry in bib_database.entries:
        citation_key = entry['ID'][:15]  # Truncate citation key to max 15 characters
        keywords = entry.get('keywords', 'No keywords')  # Use 'No keywords' if not present
        data.append([citation_key, keywords])
        data.append(["", ""])  # This creates the empty line after each entry

    return data

data = extract_keys_and_keywords(file_path)

# Output the table using tabulate
Markdown(tabulate(data, headers=['Citation Key', 'Keywords'], maxcolwidths=[20, 55]))

The keywords have been added to the bibtex file after consulting the publishers website and/or the web of science service.

\fontsize{11}{12}\selectfont